In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("../Dataset/LIAR DATASET/train.tsv", sep='\t', header=None)
valid_df = pd.read_csv("../Dataset/LIAR DATASET/valid.tsv", sep='\t', header=None)
test_df  = pd.read_csv("../Dataset/LIAR DATASET/test.tsv", sep='\t', header=None)

In [ ]:
|train_df.head()
valid_df.head()
test_df.head()||